In [1]:
!pip install pulp
from pulp import *
import pandas as pd
import numpy as np

In [2]:
items_df = pd.read_csv("https://github.com/wi3jmu/DSS1920/raw/master/Data/items.csv")
bags_df = pd.read_csv("https://github.com/wi3jmu/DSS1920/raw/master/Data/bags.csv")

In [3]:
def solve_knapsack(items_data, bags_data):
    
    weights = items_data.weight
    values = items_data.value
    items = items_data.index.tolist()

    costs = bags_data.cost
    capas = bags_data.capacity
    bags = bags_data.index.tolist()

    m = LpProblem("KnapsackProblem", LpMaximize)

    # Variables
    x = LpVariable.dicts('x',(bags, items), lowBound=0, upBound=1, cat=pulp.LpInteger)
    y = LpVariable.dicts('y',(bags), lowBound=0, upBound=1, cat=pulp.LpInteger)

    # Objective
    m += lpSum([values[i] * x[b][i] for i in items for b in bags] + [-costs[b] * y[b] for b in bags])

    # Constraints
    for b in bags:
        #m += lpSum([weights[i] * x[b][i] for i in items ]) <= capas[b]
        m += LpAffineExpression(list(map(tuple,(zip(x[b].values(), weights))))) <= capas[b]
        m += lpSum([x[b][i] for i in items]) <= y[b]*max(bags_data.capacity)

    for i in items:
        m += lpSum([x[b][i] for b in bags]) <= 1  

    m.solve(PULP_CBC_CMD(maxSeconds=60))
    print ("Status:", LpStatus[m.status])
    print("Objective = %f" % value(m.objective))

In [4]:
items_data = items_df
bags_data = bags_df

In [5]:
solve_knapsack(items_data, bags_data)

Status: Not Solved
Objective = 26063.200000


In [6]:
print(sum(items_data.weight))
print(sum(bags_data.capacity))

110146
278.66938


### Remove low value items 

In [9]:
items_data = items_df.sort_values(by=['value'], ascending=False).head(50)
bags_data = bags_df
print(sum(items_data.weight))
print(sum(bags_data.capacity))

305
278.66938


In [10]:
solve_knapsack(items_data, bags_data)

Status: Optimal
Objective = 4681.200000


### Remove heavy items

In [13]:
items_data = items_df.sort_values(by=['weight']).head(350)
bags_data = bags_df
print(sum(items_data.weight))
print(sum(bags_data.capacity))

350
278.66938


In [14]:
solve_knapsack(items_data, bags_data)

Status: Optimal
Objective = 18281.200000


### Value Density

In [15]:
items_df['valueDensity'] = items_df.apply(lambda r: (r.value/r.weight), axis=1)

In [16]:
items_df.head()

,weight,value,valueDensity
0,7,89,12.714286
1,1,58,58.000000
2,6,35,5.833333
3,8,88,11.000000
4,9,43,4.777778


In [18]:
items_data = items_df.sort_values(by=['valueDensity'], ascending=False).head(350)
bags_data = bags_df
print(sum(items_data.weight))
print(sum(bags_data.capacity))

350
278.66938


In [19]:
solve_knapsack(items_data, bags_data)

Status: Optimal
Objective = 26063.200000


### Random Subproblems

In [37]:
items_df_1, items_df_2, items_df_3, items_df_4= np.array_split(items_df, 4)
bags_df_1, bags_df_2, bags_df_3, bags_df_4 = np.array_split(bags_df, 4)

In [38]:
items_data = items_df_1
bags_data = bags_df_1

solve_knapsack(items_data, bags_data)

Status: Optimal
Objective = 5714.900000


In [39]:
items_data = items_df_2
bags_data = bags_df_2

solve_knapsack(items_data, bags_data)

Status: Optimal
Objective = 5520.600000


In [40]:
items_data = items_df_3
bags_data = bags_df_3

solve_knapsack(items_data, bags_data)

Status: Optimal
Objective = 6288.100000


In [41]:
items_data = items_df_4
bags_data = bags_df_4

solve_knapsack(items_data, bags_data)

Status: Optimal
Objective = 8476.600000


In [42]:
5714.9 + 5520.6 + 6288.1 + 8476.6

26000.199999999997